In [10]:
import netsquid as ns
import numpy as np
from netsquid.protocols import NodeProtocol
from netsquid.nodes import Node, Network
import matplotlib.pyplot as plt
from component import QuantumConnection, ClassicalConnection
from bool_function import bool_func
from verifier import V0Protocol, V1Protocol
from prover import PProtocol

In [11]:
node_v0 = Node('v0', port_names = ['quantum', 'v0p', 'v0v1'])
node_p = Node('p', port_names=['quantum', 'pv0', 'pv1'])
node_v1 = Node('v1', port_names=['v1p', 'v1v0'])
network = Network('QPV_BB84')
network.add_nodes([node_p, node_v0, node_v1])
network.add_connection(node_v1, node_v0, connection=ClassicalConnection(length=3, name='v0v1', direction='Bi'), port_name_node1='v1v0', port_name_node2='v0v1')
network.add_connection(node_v1, node_p, connection=ClassicalConnection(length=3, name='v1p', direction='Bi'), port_name_node1='v1p', port_name_node2='pv1')
network.add_connection(node_v0, node_p, connection=ClassicalConnection(length=3, name='v0p', direction='Bi'), port_name_node1='v0p', port_name_node2='pv0')
network.add_connection(node_v0, node_p, connection=QuantumConnection(name="Channel_A2B", length=3, direction='A2B'), port_name_node1='quantum', port_name_node2='quantum')

('quantum', 'quantum')

In [12]:
handler_v0 = lambda m: print(f"V0 received {m} at {ns.sim_time()}!")
handler_v1 = lambda m: print(f"V1 received {m} at {ns.sim_time()}!")
handler_vp = lambda m: print(f'p received {m} at {ns.sim_time()}!')
node_v0.ports['v0v1'].bind_input_handler(handler_v0)
node_v1.ports['v1v0'].bind_input_handler(handler_v1)
node_p.ports['quantum'].bind_input_handler(handler_vp)
q1 = ns.qubits.create_qubits(1)[0]
q2 = ns.qubits.create_qubits(1)[0]
ns.qubits.operate(q1, ns.H)
ns.qubits.operate([q1,q2], ns.CNOT)
node_v0.ports['quantum'].tx_output(q1)

node_v0.ports['v0v1'].tx_output("Hi v1!")
node_v1.ports['v1v0'].tx_output("Hi v0!")

x = ns.sim_run()
ns.sim_reset()

V0 received Message(items=['Hi v0!'], header=None, ch_delta_time=1e-05) at 1e-05!
V1 received Message(items=['Hi v1!'], header=None, ch_delta_time=1e-05) at 1e-05!
p received Message(items=[Qubit('QS#2-0')], header=None, ch_delta_time=1.5e-05) at 1.5e-05!


In [4]:
from mpi4py import MPI

In [9]:
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
print(f'size: {size}, rank: {rank}')

size: 1, rank: 0
